In [16]:
import os
import pickle
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vocabulary, Vectorizer, HeadQA, clean_words, parse_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle
from training import train, validate, evaluate, evaluator, evaluate_better

from supervised_models import BasicLSTM

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [3]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [4]:
# training_instances = parse_dataset(training)
# validation_instances = parse_dataset(validation)
# testing_instances = parse_dataset(testing)

# oversampled_training = random_oversamplig(training_instances)

# save_dataset_to_pickle('../data/training.pickle', training_instances)
# save_dataset_to_pickle('../data/validation.pickle', validation_instances)
# save_dataset_to_pickle('../data/testing.pickle', testing_instances)
# save_dataset_to_pickle('../data/oversampled_training.pickle', oversampled_training)

In [4]:
training_instances = load_dataset_from_pickle('../data/training.pickle')
validation_instances = load_dataset_from_pickle('../data/validation.pickle')
testing_instances = load_dataset_from_pickle('../data/testing.pickle')
oversampled_training = load_dataset_from_pickle('../data/oversampled_training.pickle')

In [5]:
vectorizer = Vectorizer.vectorize_training(oversampled_training)

In [6]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [7]:
trainset = HeadQA(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=30)
validset = HeadQA(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=30)
testset = HeadQA(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=30)

In [8]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [10]:
def get_optimizer(model, lr=0.01, wd=0.0):
    return torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [11]:
model = BasicLSTM(len(vocab), 64, trainset.max_length, 1, embedding_dim=100)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

In [12]:
training_results = train(model, optimizer, train_dt, valid_dt, validate, epochs=50)

C:\Users\tec005m\Anaconda3\envs\afi\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 train loss  0.5090 valid loss 0.003 and accuracy 0.7500
Epoch 1 train loss  0.5004 valid loss 0.003 and accuracy 0.7500
Epoch 2 train loss  0.4986 valid loss 0.003 and accuracy 0.7500
Epoch 3 train loss  0.4947 valid loss 0.004 and accuracy 0.7500
Epoch 4 train loss  0.4828 valid loss 0.004 and accuracy 0.7472
Epoch 5 train loss  0.4529 valid loss 0.004 and accuracy 0.7338
Epoch 6 train loss  0.4148 valid loss 0.005 and accuracy 0.7267
Epoch 7 train loss  0.3680 valid loss 0.005 and accuracy 0.7169
Epoch 8 train loss  0.3345 valid loss 0.006 and accuracy 0.7072
Epoch 9 train loss  0.3067 valid loss 0.007 and accuracy 0.7068
Epoch 10 train loss  0.2832 valid loss 0.007 and accuracy 0.7055
Epoch 11 train loss  0.2681 valid loss 0.008 and accuracy 0.6892
Epoch 12 train loss  0.2557 valid loss 0.008 and accuracy 0.6724
Epoch 13 train loss  0.2434 valid loss 0.009 and accuracy 0.6509
Epoch 14 train loss  0.2327 valid loss 0.009 and accuracy 0.6846
Epoch 15 train loss  0.2245 valid l

In [13]:
acc, points = evaluate(model, validation, trainset.encode, evaluator)
acc, points

(tensor([0.2687]), 102)

In [14]:
acc, points = evaluate(model, testing, trainset.encode, evaluator)
acc, points

(tensor([0.2509]), 10)

In [15]:
save_dataset_to_pickle('../data/train_results_lstm.pickle', training_results)
training_results = load_dataset_from_pickle('../data/train_results_lstm.pickle')

In [16]:
training_results

[[0.5090134263756764, tensor(0.0035, requires_grad=True), 0.75],
 [0.5004443045840206, tensor(0.0034, requires_grad=True), 0.75],
 [0.4986275760524244, tensor(0.0035, requires_grad=True), 0.75],
 [0.4946762611348945, tensor(0.0035, requires_grad=True), 0.75],
 [0.4827755675976535, tensor(0.0037, requires_grad=True), 0.7472426470588235],
 [0.45287849709212064, tensor(0.0043, requires_grad=True), 0.7338235294117647],
 [0.4148019633738391, tensor(0.0050, requires_grad=True), 0.7266544117647059],
 [0.36803981342229497, tensor(0.0051, requires_grad=True), 0.7169117647058824],
 [0.3345039293349507, tensor(0.0062, requires_grad=True), 0.7071691176470588],
 [0.30671779653394077, tensor(0.0072, requires_grad=True), 0.7068014705882353],
 [0.2832131722695138, tensor(0.0073, requires_grad=True), 0.705514705882353],
 [0.26806766588285746, tensor(0.0081, requires_grad=True), 0.6891544117647059],
 [0.25569949992090824, tensor(0.0084, requires_grad=True), 0.6724264705882353],
 [0.24343141013060707, te

In [17]:
model_path = os.getcwd() + '/trained_models/basic_lstm_sig'
torch.save(model.state_dict(), model_path)

In [9]:
model = BasicLSTM(len(vocab), 64, trainset.max_length, 1, embedding_dim=100)
model.load_state_dict(torch.load(os.getcwd() + '/trained_models/basic_lstm_sig'))
model.eval()

BasicLSTM(
  (embeddings): Embedding(20403, 100, padding_idx=0)
  (lstm): LSTM(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [17]:
acc, points, acc_list, points_list = evaluate_better(model, validation, trainset.encode, evaluator)
acc, points, acc_list, points_list

(0.26881942,
 17.0,
 [tensor(0.2965),
  tensor(0.2217),
  tensor(0.2533),
  tensor(0.2554),
  tensor(0.3009),
  tensor(0.2851)],
 [42, -26, 3, 5, 46, 32])

In [18]:
acc, points, acc_list, points_list = evaluate_better(model, testing, trainset.encode, evaluator)
acc, points, acc_list, points_list

(0.25091484,
 0.8333333333333334,
 [tensor(0.2325),
  tensor(0.2601),
  tensor(0.2544),
  tensor(0.2414),
  tensor(0.2696),
  tensor(0.2597),
  tensor(0.2345),
  tensor(0.2802),
  tensor(0.2489),
  tensor(0.2035),
  tensor(0.2356),
  tensor(0.2907)],
 [-16, 9, 4, -8, 18, 9, -14, 28, -1, -43, -13, 37])